# Select Your IPTS 

In [ ]:
from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

# Description 

This notebook performs the normalization of a set of images (tiff or fits) by OBs. 

The steps are:
 - select your **sample images**
 - select your **ob images**
 - select your **df images** (optional)
 - all the data will be loaded
 - preview of the sample and ob
 - selection of a **background** region in the sample data
 - normalization is performed
 - export of the normalized data

# Python Import

In [ ]:
from __code.normalization_new_ui import *

In [ ]:
%matplotlib notebook

# Loading Images 

In [ ]:
files = Files()
sample_panel = SampleSelectionPanel(working_dir=system.System.get_working_dir())
sample_panel.init_ui(files=files)
wizard = WizardPanel(sample_panel=sample_panel)

# Preview Data

## sample

In [ ]:
o_norm = sample_panel.o_norm
o_norm.plot_images(data_type='sample')

## ob

In [ ]:
o_norm.plot_images(data_type='ob')

# Select Background Region 

In [ ]:
o_norm.with_or_without_roi()

In [ ]:
o_norm.select_sample_roi()

# Normalization

In [ ]:
o_norm.run_normalization()

# Export 

In [ ]:
o_norm.select_export_folder()

In [ ]:
o_norm.export()